# Linear Regression Practice 

This noteook will implement gradient descent using multivariate linear regression to predict wine quality

In [32]:
# Scientific and vector computation for python
import numpy as np

# Plotting library
from tabulate import tabulate
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D  # needed to plot 3-D surfaces

# tells matplotlib to embed plots within the notebook
%matplotlib inline

### Loading the Data and perform feature scaling

First, we load the data and use feature normalization to ensure gradient descent converges much more quickly

In [39]:
# Load data
data = np.loadtxt(open("data/wineQuality.csv", "rb"), delimiter=",", skiprows=1)


# Create feature matrix and output variables
# Here, X denotes our feature matrix and y is the output
X = data[:, :-1]
y = data[:, -1]

# print out some data points
sample = data[:10, :]
table = [column for column in sample]
print(tabulate(table, headers=["X[0]", "X[1]", "X[2]", "X[3]", "X[4]", "X[5]", "X[6]", "X[7]", "X[8]", "X[9]", "X[10]", "quality (1-10)" ]))

print("Legend")

  X[0]    X[1]    X[2]    X[3]    X[4]    X[5]    X[6]    X[7]    X[8]    X[9]    X[10]    quality (1-10)
------  ------  ------  ------  ------  ------  ------  ------  ------  ------  -------  ----------------
   7.4    0.7     0        1.9   0.076      11      34  0.9978    3.51    0.56      9.4                 5
   7.8    0.88    0        2.6   0.098      25      67  0.9968    3.2     0.68      9.8                 5
   7.8    0.76    0.04     2.3   0.092      15      54  0.997     3.26    0.65      9.8                 5
  11.2    0.28    0.56     1.9   0.075      17      60  0.998     3.16    0.58      9.8                 6
   7.4    0.7     0        1.9   0.076      11      34  0.9978    3.51    0.56      9.4                 5
   7.4    0.66    0        1.8   0.075      13      40  0.9978    3.51    0.56      9.4                 5
   7.9    0.6     0.06     1.6   0.069      15      59  0.9964    3.3     0.46      9.4                 5
   7.3    0.65    0        1.2   0.065      15